1.Import Libraries

In [14]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler

# Hide warnings for cleaner output
pd.set_option('mode.chained_assignment', None)


2.Load RAW Dataset

In [11]:
import os

# Check if the file exists
print(os.path.exists('data/raw/retail_store_sales.csv'))


True


In [10]:
import pandas as pd

raw_data_path = r'data/raw/retail_store_sales.csv' 
df = pd.read_csv(raw_data_path)
print("First 5 rows:")
display(df.head())


First 5 rows:


,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


3.Examine Dataset

In [12]:
print("Dataset shape:", df.shape)
display(df.info())
display(df.describe())

print("Missing values per column:")
print(df.isnull().sum())

print("Number of duplicate rows:", df.duplicated().sum())

Dataset shape: (12575, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB


None

,Price Per Unit,Quantity,Total Spent
count,11966.000000,11971.000000,11971.000000
mean,23.365912,5.536380,129.652577
std,10.743519,2.857883,94.750697
min,5.000000,1.000000,5.000000
25%,14.000000,3.000000,51.000000
50%,23.000000,6.000000,108.500000
75%,33.500000,8.000000,192.000000
max,41.000000,10.000000,410.000000


Missing values per column:
Transaction ID         0
Customer ID            0
Category               0
Item                1213
Price Per Unit       609
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64
Number of duplicate rows: 0


4.Handle Missing Values and Save Cleaned Dataset

In [17]:

# Drop rows with missing 'Item'
df = df.dropna(subset=['Item'])

# Fill numeric missing values with median
num_cols = ['Price Per Unit', 'Quantity', 'Total Spent']
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# Fill categorical missing values with mode
df['Discount Applied'] = df['Discount Applied'].fillna(df['Discount Applied'].mode()[0])

# Remove duplicate rows
df = df.drop_duplicates()

# Ensure the processed directory exists
processed_dir = 'data/processed/'
os.makedirs(processed_dir, exist_ok=True)

# Save the cleaned dataset
output_path = os.path.join(processed_dir, 'cleaned1_clustering.csv')
df.to_csv(output_path, index=False)

# Confirmation prints
print("Missing values after cleaning:\n", df.isnull().sum())
print("Number of duplicate rows after cleaning:", df.duplicated().sum())
print(f"Cleaned dataset successfully saved at: {output_path}")
print("Check existence:", os.path.exists(output_path))


Missing values after cleaning:
 Transaction ID      0
Customer ID         0
Category            0
Item                0
Price Per Unit      0
Quantity            0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
Discount Applied    0
dtype: int64
Number of duplicate rows after cleaning: 0
Cleaned dataset successfully saved at: data/processed/cleaned1_clustering.csv
Check existence: True


5.Outlier Detection & Removal using IQR

In [18]:

def remove_outliers_iqr(data, col):
    """
    Remove outliers from a numeric column using the IQR method.
    """
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    filtered_data = data[(data[col] >= lower) & (data[col] <= upper)]
    removed_count = data.shape[0] - filtered_data.shape[0]
    print(f"{removed_count} outliers removed from '{col}'")
    return filtered_data

# Apply outlier removal for each numeric column
num_cols = ['Price Per Unit', 'Quantity', 'Total Spent']
for col in num_cols:
    df = remove_outliers_iqr(df, col)

# Print final shape after outlier removal
print("Dataset shape after outlier removal:", df.shape)

# Save the dataset after outlier removal (optional)
output_path_outliers = os.path.join('data/processed', 'cleaned2_no_outliers.csv')
df.to_csv(output_path_outliers, index=False)
print(f"Dataset after outlier removal saved at: {output_path_outliers}")
print("Check existence:", os.path.exists(output_path_outliers))


0 outliers removed from 'Price Per Unit'
0 outliers removed from 'Quantity'
56 outliers removed from 'Total Spent'
Dataset shape after outlier removal: (11306, 11)
Dataset after outlier removal saved at: data/processed\cleaned2_no_outliers.csv
Check existence: True


6.Encoding Categorical Columns

In [19]:

categorical_cols = ['Category', 'Item', 'Payment Method', 'Location', 'Discount Applied']

# Apply one-hot encoding
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Print confirmation
print("Original number of columns:", df.shape[1])
print("Number of columns after encoding:", df_encoded.shape[1])
print("Sample of encoded columns:")
display(df_encoded.head())

# save encoded dataset
encoded_output_path = 'data/processed/encoded_clustering.csv'
df_encoded.to_csv(encoded_output_path, index=False)
print(f"Encoded dataset saved at: {encoded_output_path}")
print("Check existence:", os.path.exists(encoded_output_path))


Original number of columns: 11
Number of columns after encoding: 216
Sample of encoded columns:


,Transaction ID,Customer ID,Price Per Unit,Quantity,Total Spent,Transaction Date,Category_Butchers,Category_Computers and electric accessories,Category_Electric household essentials,Category_Food,...,Item_Item_9_CEA,Item_Item_9_EHE,Item_Item_9_FOOD,Item_Item_9_FUR,Item_Item_9_MILK,Item_Item_9_PAT,Payment Method_Credit Card,Payment Method_Digital Wallet,Location_Online,Discount Applied_True
0,TXN_6867343,CUST_09,18.5,10.0,185.0,2024-04-08,False,False,False,False,...,False,False,False,False,False,False,False,True,True,True
1,TXN_3731986,CUST_22,29.0,9.0,261.0,2023-07-23,False,False,False,False,...,False,False,False,False,False,False,False,True,True,True
2,TXN_9303719,CUST_02,21.5,2.0,43.0,2022-10-05,True,False,False,False,...,False,False,False,False,False,False,True,False,True,False
3,TXN_9458126,CUST_06,27.5,9.0,247.5,2022-05-07,False,False,False,False,...,False,False,False,False,False,False,True,False,True,True
4,TXN_4575373,CUST_05,12.5,7.0,87.5,2022-10-02,False,False,False,True,...,False,False,False,False,False,False,False,True,True,False


Encoded dataset saved at: data/processed/encoded_clustering.csv
Check existence: True


7.Feature Scaling

In [21]:

from sklearn.preprocessing import StandardScaler

# Columns to scale (exclude non-numeric original columns)
cols_to_scale = df_encoded.select_dtypes(include=[np.number]).columns

scaler = StandardScaler()

df_scaled = df_encoded.copy()
df_scaled[cols_to_scale] = scaler.fit_transform(df_encoded[cols_to_scale])

# Print confirmation
print("First 5 rows after scaling numeric columns:")
display(df_scaled.head())

print("Check min and max of scaled columns:")
print(df_scaled[cols_to_scale].describe().loc[['min','max']])

# Save final preprocessed dataset
final_output_path = 'data/processed/final_preprocessed_clustering.csv'
df_scaled.to_csv(final_output_path, index=False)

print(f"Final preprocessed dataset saved at: {final_output_path}")
print("Check existence:", os.path.exists(final_output_path))



First 5 rows after scaling numeric columns:


,Transaction ID,Customer ID,Price Per Unit,Quantity,Total Spent,Transaction Date,Category_Butchers,Category_Computers and electric accessories,Category_Electric household essentials,Category_Food,...,Item_Item_9_CEA,Item_Item_9_EHE,Item_Item_9_FOOD,Item_Item_9_FUR,Item_Item_9_MILK,Item_Item_9_PAT,Payment Method_Credit Card,Payment Method_Digital Wallet,Location_Online,Discount Applied_True
0,TXN_6867343,CUST_09,-0.446166,1.577342,0.611436,2024-04-08,False,False,False,False,...,False,False,False,False,False,False,False,True,True,True
1,TXN_3731986,CUST_22,0.536071,1.225813,1.430332,2023-07-23,False,False,False,False,...,False,False,False,False,False,False,False,True,True,True
2,TXN_9303719,CUST_02,-0.165526,-1.234892,-0.918606,2022-10-05,True,False,False,False,...,False,False,False,False,False,False,True,False,True,False
3,TXN_9458126,CUST_06,0.395752,1.225813,1.284870,2022-05-07,False,False,False,False,...,False,False,False,False,False,False,True,False,True,True
4,TXN_4575373,CUST_05,-1.007444,0.522754,-0.439121,2022-10-02,False,False,False,True,...,False,False,False,False,False,False,False,True,True,False


Check min and max of scaled columns:
     Price Per Unit  Quantity  Total Spent
min       -1.709042 -1.586421    -1.328054
max        1.658628  1.577342     2.874174
Final preprocessed dataset saved at: data/processed/final_preprocessed_clustering.csv
Check existence: True
